# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'ADSNAQ',
 'calculationPrice': 'iexlasttrade',
 'open': None,
 'openTime': None,
 'openSource': 'ffolciai',
 'close': None,
 'closeTime': None,
 'closeSource': 'faciolfi',
 'high': 466.25,
 'highTime': 1629149140321,
 'highSource': 'i 1e nylaipce5dudter me',
 'low': 469.31,
 'lowTime': 1628081314759,
 'lowSource': '5nyamte pice duee1dlir ',
 'latestPrice': 476.94,
 'latestSource': 'IEX Last Trade',
 'latestTime': 'August 13, 2020',
 'latestUpdate': 1661144800199,
 'latestVolume': 52501376,
 'iexRealtimePrice': 471.48,
 'iexRealtimeSize': 103,
 'iexLastUpdated': 1602768699036,
 'delayedPrice': None,
 'delayedPriceTime': None,
 'oddLotDelayedPrice': None,
 'oddLotDelayedPriceTime': None,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 458.67,
 'previousVolume': 41491757,
 'change': 7.78,
 'changePercent': 0.01754,
 'volume': 52148835,
 'iexM

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [4]:
pe_ratio = data['peRatio']
pe_ratio

34.95

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,99.150,45.02,N/A
1,AAL,13.400,-1.64,N/A
2,AAP,158.731,29.05,N/A
3,AAPL,468.690,35.35,N/A
4,ABBV,94.860,21.07,N/A
...,...,...,...,...
500,YUM,94.110,27.44,N/A
501,ZBH,138.200,711.6,N/A
502,ZBRA,299.797,31.84,N/A
503,ZION,35.730,13.2,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,65.133,2389.8,N/A
1,FIS,145.503,1544.42,N/A
2,GPC,97.240,948.15,N/A
3,ZBH,138.200,711.6,N/A
4,HLT,86.500,612.19,N/A
5,STZ,182.650,603.35,N/A
6,HRB,14.919,555.16,N/A
7,GLW,33.190,253.28,N/A
8,IQV,170.830,241.94,N/A
9,KMI,14.463,211.43,N/A


## Calculating the Number of Shares to Buy
We now need to calculate the number of shares we need to buy. 

To do this, we will use the `portfolio_input` function that we created in our momentum project.

I have included this function below.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

Use the `portfolio_input` function to accept a `portfolio_size` variable from the user of this script.

In [9]:
portfolio_input()

Enter the value of your portfolio:1000000


You can now use the global `portfolio_size` variable to calculate the number of shares that our strategy should purchase.

In [10]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,65.133,2389.8,301
1,FIS,145.503,1544.42,134
2,GPC,97.240,948.15,201
3,ZBH,138.200,711.6,141
4,HLT,86.500,612.19,226
5,STZ,182.650,603.35,107
6,HRB,14.919,555.16,1314
7,GLW,33.190,253.28,590
8,IQV,170.830,241.94,114
9,KMI,14.463,211.43,1355


## Building a Better (and More Realistic) Momentum Strategy
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. We'll start by calculating each data point from scratch.

In [11]:
symbol = 'AAPL'
batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

Let's move on to building our DataFrame. You'll notice that I use the abbreviation `rv` often. It stands for `robust value`, which is what we'll call this sophisticated strategy moving forward.

In [12]:
rv_columns = [
                'Ticker', 
                'Price', 
                'Number of Shares to Buy', 
                'Price-to-Earnings Ratio',
                'PE Percentile',
                'Price-to-Book Ratio',
                'PB Percentile',
                'EV/EBITDA',
                'EV/EBITDA Percentile',
                'EV/GP',
                'EV/GP Percentile',
                'RV Score'
                ]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        
        pe_ratio = data[symbol]['quote']['peRatio']
        pb_ratio = data[symbol]['advanced-stats']['priceToBook']
        ps_ratio = data[symbol]['advanced-stats']['priceToSales']
        
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
            
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
                                        pd.Series([
                                                    symbol, 
                                                    data[symbol]['quote']['latestPrice'], 
                                                    'N/A', 
                                                    pe_ratio,
                                                    'N/A',
                                                    pb_ratio,
                                                    'N/A',
                                                    ev_to_ebitda,
                                                    'N/A',
                                                    ev_to_gross_profit,
                                                    'N/A',
                                                    'N/A'
                                                   ], 
                                                  index = rv_columns), 
                                        ignore_index = True)
        

rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,101.440,N/A,46.11,N/A,6.371653,N/A,26.865217,N/A,11.047035,N/A,N/A
1,AAL,13.650,N/A,-1.69,N/A,-61.060616,N/A,5.916299,N/A,3.212977,N/A,N/A
2,AAP,160.657,N/A,28.71,N/A,3.127808,N/A,15.224044,N/A,3.293698,N/A,N/A
3,AAPL,465.400,N/A,35.64,N/A,22.463973,N/A,26.225125,N/A,20.249733,N/A,N/A
4,ABBV,96.400,N/A,20.12,N/A,-20.949970,N/A,12.255004,N/A,7.568305,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.130,N/A,27.33,N/A,-3.527164,N/A,18.770575,N/A,14.154888,N/A,N/A
501,ZBH,142.020,N/A,713.35,N/A,2.336419,N/A,16.040361,N/A,7.100027,N/A,N/A
502,ZBRA,296.474,N/A,31.15,N/A,8.570317,N/A,20.089628,N/A,8.663448,N/A,N/A
503,ZION,34.930,N/A,13.41,N/A,0.794338,N/A,NaN,N/A,NaN,N/A,N/A


In [13]:
print(data[symbol]['advanced-stats'])

{'week52change': 0.287645, 'week52high': 164.37, 'week52low': 91.88, 'marketcap': 76883163532, 'employees': 10723, 'day200MovingAvg': 134.7, 'day50MovingAvg': 144.77, 'float': 486186715, 'avg10Volume': 1614074, 'avg30Volume': 1483446.97, 'ttmEPS': 3.4931, 'ttmDividendRate': 0.788, 'companyName': 'Zoetis, Inc.', 'sharesOutstanding': 489251656, 'maxChangePercent': 3.4099, 'year5ChangePercent': 1.683, 'year2ChangePercent': 0.6103, 'year1ChangePercent': 0.28579, 'ytdChangePercent': 0.194893, 'month6ChangePercent': 0.103322, 'month3ChangePercent': 0.298621, 'month1ChangePercent': 0.15893, 'day30ChangePercent': 0.162359, 'day5ChangePercent': -0.012505, 'nextDividendDate': None, 'dividendYield': 0.00491546129743793, 'nextEarningsDate': '2020-10-10', 'exDividendDate': '2020-07-02', 'peRatio': 46.98, 'beta': 0.9892719427062117, 'totalCash': 1962438955, 'currentDebt': 559973043, 'revenue': 6535321056, 'grossProfit': 4163952351, 'totalRevenue': 6399267878, 'EBITDA': 2512960654, 'revenuePerShare':

## Dealing With Missing Data in Our DataFrame

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using. 

You can use pandas' `isnull` method to identify missing data:

In [14]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
17,AFL,37.863,N/A,9.82,N/A,0.922686,N/A,NaN,N/A,NaN,N/A,N/A
18,AIG,31.470,N/A,-5.85,N/A,0.415183,N/A,NaN,N/A,NaN,N/A,N/A
20,AIZ,126.940,N/A,19.72,N/A,1.322977,N/A,NaN,N/A,NaN,N/A,N/A
26,ALL,100.500,N/A,7.2,N/A,1.209077,N/A,NaN,N/A,NaN,N/A,N/A
39,ANTM,288.000,N/A,12.49,N/A,2.241986,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,194.932,N/A,26.23,N/A,13.525891,N/A,18.528564,N/A,NaN,N/A,N/A
56,BAC,27.062,N/A,12.95,N/A,0.877889,N/A,NaN,N/A,NaN,N/A,N/A
64,BK,37.540,N/A,8.19,N/A,0.808925,N/A,NaN,N/A,NaN,N/A,N/A
65,BKNG,1856.990,N/A,30.68,N/A,12.775506,N/A,12.302080,N/A,NaN,N/A,N/A
75,C,53.123,N/A,9.37,N/A,0.574859,N/A,NaN,N/A,NaN,N/A,N/A


Dealing with missing data is an important topic in data science.

There are two main approaches:

* Drop missing data from the data set (pandas' `dropna` method is useful here)
* Replace missing data with a new value (pandas' `fillna` method is useful here)

In this tutorial, we will replace missing data with the average non-`NaN` data point from that column. 

Here is the code to do this:

In [15]:
for column in ['Price-to-Earnings Ratio', 'EV/EBITDA', 'EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [16]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


## Calculating Value Percentiles

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* EV/EBITDA
* EV/GP

Here's how we'll do this:

In [17]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.837624
1      0.114851
2      0.619802
3      0.742574
4      0.409901
         ...   
500    0.574257
501    0.994059
502    0.679208
503    0.265347
504    0.845545
Name: PE Percentile, Length: 505, dtype: object
0       0.734653
1      0.0158416
2       0.508911
3       0.940594
4      0.0257426
         ...    
500     0.049505
501     0.409901
502     0.809901
503     0.134653
504     0.954455
Name: PB Percentile, Length: 505, dtype: object
0       0.885149
1      0.0712871
2        0.50099
3       0.873267
4       0.342574
         ...    
500     0.724752
501     0.526733
502     0.784158
503     0.671287
504     0.924752
Name: EV/EBITDA Percentile, Length: 505, dtype: object
0       0.538614
1      0.0613861
2      0.0712871
3       0.879208
4       0.330693
         ...    
500     0.750495
501     0.308911
502     0.407921
503     0.642574
504     0.859406
Name: EV/GP Percentile, Length: 505, dtype: object


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,101.440,N/A,46.11,0.837624,6.371653,0.734653,26.865217,0.885149,11.047035,0.538614,N/A
1,AAL,13.650,N/A,-1.69,0.114851,-61.060616,0.0158416,5.916299,0.0712871,3.212977,0.0613861,N/A
2,AAP,160.657,N/A,28.71,0.619802,3.127808,0.508911,15.224044,0.50099,3.293698,0.0712871,N/A
3,AAPL,465.400,N/A,35.64,0.742574,22.463973,0.940594,26.225125,0.873267,20.249733,0.879208,N/A
4,ABBV,96.400,N/A,20.12,0.409901,-20.949970,0.0257426,12.255004,0.342574,7.568305,0.330693,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.130,N/A,27.33,0.574257,-3.527164,0.049505,18.770575,0.724752,14.154888,0.750495,N/A
501,ZBH,142.020,N/A,713.35,0.994059,2.336419,0.409901,16.040361,0.526733,7.100027,0.308911,N/A
502,ZBRA,296.474,N/A,31.15,0.679208,8.570317,0.809901,20.089628,0.784158,8.663448,0.407921,N/A
503,ZION,34.930,N/A,13.41,0.265347,0.794338,0.134653,18.736507,0.671287,12.222257,0.642574,N/A


## Calculating the RV Score
We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [18]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,101.440,N/A,46.11,0.837624,6.371653,0.734653,26.865217,0.885149,11.047035,0.538614,0.74901
1,AAL,13.650,N/A,-1.69,0.114851,-61.060616,0.0158416,5.916299,0.0712871,3.212977,0.0613861,0.0658416
2,AAP,160.657,N/A,28.71,0.619802,3.127808,0.508911,15.224044,0.50099,3.293698,0.0712871,0.425248
3,AAPL,465.400,N/A,35.64,0.742574,22.463973,0.940594,26.225125,0.873267,20.249733,0.879208,0.858911
4,ABBV,96.400,N/A,20.12,0.409901,-20.949970,0.0257426,12.255004,0.342574,7.568305,0.330693,0.277228
...,...,...,...,...,...,...,...,...,...,...,...,...
500,YUM,94.130,N/A,27.33,0.574257,-3.527164,0.049505,18.770575,0.724752,14.154888,0.750495,0.524752
501,ZBH,142.020,N/A,713.35,0.994059,2.336419,0.409901,16.040361,0.526733,7.100027,0.308911,0.559901
502,ZBRA,296.474,N/A,31.15,0.679208,8.570317,0.809901,20.089628,0.784158,8.663448,0.407921,0.670297
503,ZION,34.930,N/A,13.41,0.265347,0.794338,0.134653,18.736507,0.671287,12.222257,0.642574,0.428465


## Selecting the 50 Best Momentum Stocks¶

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [19]:
rv_dataframe.sort_values(by = 'RV Score', ascending = False)
rv_dataframe = rv_dataframe[:51]

## Calculating the Number of Shares to Buy
We'll use the `portfolio_input` function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [20]:
portfolio_input()

Enter the value of your portfolio:1000000


In [21]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,101.440,193,46.11,0.837624,6.371653,0.734653,26.865217,0.885149,11.047035,0.538614,0.74901
1,AAL,13.650,1436,-1.69,0.114851,-61.060616,0.0158416,5.916299,0.0712871,3.212977,0.0613861,0.0658416
2,AAP,160.657,122,28.71,0.619802,3.127808,0.508911,15.224044,0.50099,3.293698,0.0712871,0.425248
3,AAPL,465.400,42,35.64,0.742574,22.463973,0.940594,26.225125,0.873267,20.249733,0.879208,0.858911
4,ABBV,96.400,203,20.12,0.409901,-20.949970,0.0257426,12.255004,0.342574,7.568305,0.330693,0.277228
5,ABC,105.000,186,13.92,0.281188,7.425222,0.764356,9.480171,0.219802,4.969894,0.172277,0.359406
6,ABMD,314.470,62,89.86,0.948515,13.361921,0.891089,47.614051,0.962376,18.228422,0.851485,0.913366
7,ABT,106.100,184,60.50,0.885149,5.839572,0.718812,24.744146,0.849505,11.243672,0.548515,0.750495
8,ACN,234.570,83,30.60,0.665347,10.439130,0.843564,20.104985,0.786139,10.768452,0.530693,0.706436
9,ADBE,458.070,42,61.09,0.887129,21.565043,0.934653,53.256840,0.972277,22.074138,0.89505,0.922277


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [22]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File
You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

* String format for tickers
* \$XX.XX format for stock prices
* \$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase
* Boolean formats with 1 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [23]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

boolean_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )